# TruthLens Modelling - Phase 1 : Binary Classification
The aim of phase 1 is to classify text into real or fake. "Fake" content will move to phase 2 for multiclass classification, so essentially what we are trying to do with this stage is to filter out real news.

The dataset used is the Misinformation & Fake News text dataset, which has already been cleaned and preprocessed (see "TruthLens Data Cleaning" notebook).

In [1]:
#imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import csv
import random
import pickle
pd.set_option('display.max_colwidth', None)

### Feature Extraction Using TF-IDF, n-grams and readability metrics
In this section I will use a TfidfVectorizer to extract features from the training data. I will then combine these with the scaled readability metrics into one horizontally stacked feature matrix that I will use for modelling.

In [3]:
#load data
df = pd.read_csv('Data/phase1_final_clean.csv')
df = df.reset_index(drop=True)
print(df.head(1))

In [ ]:
#TF-IDF feature extraction with n-grams
start_time = time.time()
#replace NaN values with an empty string to resolve NaN ValueError
df['content_lemma_nostop'] = df['content_lemma_nostop'].fillna('')
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df['content_lemma_nostop'])
#get pre-calculated readability features
readability_features = df[['word_count', 'sentence_count', 'flesch_reading_ease']].values
#standardise readability features
scaler = StandardScaler()
readability_scaled = scaler.fit_transform(readability_features)
#convert to a sparse matrix
readability_sparse = csr_matrix(readability_scaled)
#combine TF-IDF features with the readability metrics
X_combined = hstack([X_tfidf, readability_sparse])

y = df['label']
print("Feature extraction: {:.4f} seconds".format(time.time() - start_time))

Feature extraction: 256.7663 seconds


### Split dataset

In [ ]:
#retain the indices as we need these for looking up explanations later
train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=999)
#split X and y using the train/test indices
X_train = X_combined[train_indices]
X_test = X_combined[test_indices]
y_train = y.iloc[train_indices]
y_test = y.iloc[test_indices]

### Phase 1 Modelling
I will test three different models on the data to see which is best. The models to be tested are:
- Logistic Regression
- Random Forest
- Support Vector Machine (SVM)

#### Logistic Regression

In [ ]:
start_time = time.time()
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Fit Logistic Regression model: {:.4f} seconds".format(time.time() - start_time))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fit Logistic Regression model: 6.0120 seconds
Accuracy: 0.942295457437333
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93      6931
           1       0.94      0.95      0.95      8787

    accuracy                           0.94     15718
   macro avg       0.94      0.94      0.94     15718
weighted avg       0.94      0.94      0.94     15718



#### Random Forest

In [ ]:
start_time = time.time()
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
print("Fit Random Forest model: {:.4f} seconds".format(time.time() - start_time))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

Fit Random Forest model: 347.0968 seconds
Random Forest Accuracy: 0.9523476269245451
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.95      6931
           1       0.95      0.96      0.96      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718



#### Support Vector Machine (SVM)

In [ ]:
start_time = time.time()
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print("Fit SVM model: {:.4f} seconds".format(time.time() - start_time))
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("SVM Classification Report:\n", classification_report(y_test, svm_pred))

Fit SVM model: 4252.7811 seconds
SVM Accuracy: 0.9504389871484922
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6931
           1       0.95      0.96      0.96      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718



#### Conclusion
For phase 1 to be successful according to our success metrics, we want an accuracy of at least 90%. All three algorithms tested have performed better than that.

Random Forest and SVM both had an accuracy of 0.95 which is impressive. Logistic regression was slightly lower with an accuracy of 0.94. However, when computational efficiency is taken into account, Logistic regression is the preferred model. It took just over 6 seconds to fit the Logistic regression model to our data, compared to 347 seconds (almost 6 minutes) for random forest, and a whooping 4252 seconds (just under 71 minutes) for SVM.

Next we will test what column works best - content, content_lemma (which is lemmatised) or content_lemma_nostop (which is lemmatised and additionally has stopwords and punctuation removed.)

### Column selection

In [ ]:
def evaluate_text_representation(text_column, df):
    print("Evaluating model on ",text_column)
    start_time = time.time()
    #replace NaN values with an empty string to resolve NaN ValueError
    df[text_column] = df[text_column].fillna('')

    #get TF-IDF features for the chosen text column
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
    X_tfidf = vectorizer.fit_transform(df[text_column])

    #get pre-calculated readability features
    readability_features = df[['word_count', 'sentence_count', 'flesch_reading_ease']].values
    scaler = StandardScaler()
    #standardise readability features
    readability_scaled = scaler.fit_transform(readability_features)
    #convert to a sparse matrix
    readability_sparse = csr_matrix(readability_scaled)

    #combine TF-IDF features with the readability metrics
    X_combined = hstack([X_tfidf, readability_sparse])

    y = df['label']

    #split the data into training and testing sets using the dataframe index
    train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=999)
    X_train = X_combined[train_indices]
    X_test = X_combined[test_indices]
    y_train = y.iloc[train_indices]
    y_test = y.iloc[test_indices]

    #train a simple Logistic Regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed_time = time.time() - start_time

    # Evaluate and print the results
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"Results for '{text_column}':")
    print("Time elapsed: {:.4f} seconds".format(elapsed_time))
    print("Accuracy:", accuracy)
    print("Classification Report:\n", report)
    print("-" * 50)

    return accuracy, model, vectorizer

In [ ]:
#columns to test
text_columns = ['content', 'content_lemma', 'content_lemma_nostop']
results = {}

#run the evaluation for each column
for col in text_columns:
    acc, model, vectorizer = evaluate_text_representation(col, df)
    results[col] = {'accuracy': acc, 'model': model, 'vectorizer': vectorizer}

#identify the best performing model based on accuracy
best_column = max(results, key=lambda x: results[x]['accuracy'])
best_result = results[best_column]

#print a summary of the accuracies
print("Summary of accuracies:")
for col, result in results.items():
    print(f"{col}: {result['accuracy']:.4f}")

print(f"\nBest performing column: {best_column} with accuracy: {best_result['accuracy']:.4f}")

#set the best model
best_model = best_result['model']

Evaluating model on  content
Results for 'content':
Time elapsed: 399.0861 seconds
Accuracy: 0.9401323323578064
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      6931
           1       0.94      0.95      0.95      8787

    accuracy                           0.94     15718
   macro avg       0.94      0.94      0.94     15718
weighted avg       0.94      0.94      0.94     15718

--------------------------------------------------
Evaluating model on  content_lemma
Results for 'content_lemma':
Time elapsed: 355.7318 seconds
Accuracy: 0.9481486194172286
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6931
           1       0.95      0.96      0.95      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718

-----------

#### Results
As we can see from the output above, the classifier performed best on the lemmatised column, with an accuracy of 0.9481.

Next we will use some tuning to try and increase the accuracy of our logistic regression model.

### Tweak model

In [4]:
#This code is repeated from the top of the file - look into making this a fucntion to reduce repetition
#Also I've just calculated most of this in the previous function - see if I can maybe get it from there instead of recalculating
start_time = time.time()
#replace NaN values with an empty string to resolve NaN ValueError
df['content_lemma'] = df['content_lemma'].fillna('')
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df['content_lemma'])
#get pre-calculated readability features
readability_features = df[['word_count', 'sentence_count', 'flesch_reading_ease']].values
#standardise readability features
scaler = StandardScaler()
readability_scaled = scaler.fit_transform(readability_features)
#convert to a sparse matrix
readability_sparse = csr_matrix(readability_scaled)
#combine TF-IDF features with the readability metrics
X_combined = hstack([X_tfidf, readability_sparse])

y = df['label']
print("Feature extraction: {:.4f} seconds".format(time.time() - start_time))



Feature extraction: 301.6178 seconds


In [5]:
#retain the indices as we need these for looking up explanations later
train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=999)
#split X and y using the train/test indices
X_train = X_combined[train_indices]
X_test = X_combined[test_indices]
y_train = y.iloc[train_indices]
y_test = y.iloc[test_indices]

In [6]:
start_time = time.time()
#we're going to test for the best combo of regularisation type (l1 or l2), C values, and solvers
param_grid = [
    {
        #l1 regularisation
        'penalty': ['l1'],
        'C': [0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    },
    {
        #l2 regularization
        'penalty': ['l2'],
        'C': [0.1, 1, 10],
        'solver': ['saga', 'sag']
    }
]

#set up the grid search
grid = GridSearchCV(LogisticRegression(max_iter=2000), param_grid, cv=3, n_jobs = -1)
grid.fit(X_train, y_train)

#print the best parameters and best score, and the time it took
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)
print("Grid Search: {:.4f} seconds".format(time.time() - start_time))

Best parameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Best cross-validation score: 0.954254805167687
Grid Search: 15976.0700 seconds


#### Tuning Results
As we can see from the above results XXXX

### Create and save final Phase 1 model

In [ ]:
#instantiate the final model with the best hyperparameters as found by the grid search
final_model = LogisticRegression(C=10, penalty='l2', solver='sag', max_iter=2000)

#fit the model
final_model.fit(X_train, y_train)

#evaluate on the test set
final_predictions = final_model.predict(X_test)
print("Final Model Accuracy:", accuracy_score(y_test, final_predictions))
print("Final Model Classification Report:\n", classification_report(y_test, final_predictions))

In [ ]:
#save our model
with open('final_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

### Validate model
Extra data from https://www.kaggle.com/datasets/stevenpeutz/misinformation-fake-news-text-dataset-79k

In [ ]:
#load in validation dataset
external_df = pd.read_csv('Data/extra_data_final_clean.csv')
external_df = external_df.dropna(subset=['content_lemma_nostop']).reset_index(drop=True)
external_text = external_df['content_lemma_nostop']
external_labels = external_df['label']

# Transform the external data using the already fitted vectorizer (or pipeline)
X_external = vectorizer.transform(external_text)

# Predict using the trained model
external_predictions_lr = final_model.predict(X_external)

# Evaluate the performance on the external dataset
print("External Dataset Classification Report:\n", classification_report(external_labels, external_predictions_lr))
